<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_rnd2d_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2次元データのロジスティック回帰（logistic regression of a 2D dataset）

----

氏名：

学生番号：

----
基本課題（必須）

    1. 「★scikit-learnを用いたロジスティック回帰」まで実行すると，どのような数値と図が出力されますか．
       また，表示される数値と図から何が言えますか．特に，Example 2とExample 3について記述してください．

（ここに回答を書いてください）



    2. 「★scikit-learnを用いたロジスティック回帰」で使用している model.fit(X, y) と
        model.decision_function(X) はそれぞれ何をする関数ですか．

（ここに回答を書いてください）



    3. 最急降下法によるロジスティック回帰を実装してください．

（回答はこのファイルの後半「★最急降下法によるロジスティック回帰の実装」に書いてください）



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）



----
発展課題（任意）がこのノートブックの後半にあります．

In [ ]:
#@title 2次元に2クラスを描く `plot2cls` を定義します（理解不要）．
!wget -q -N https://gist.githubusercontent.com/tsakai-g/1920bc185be0c497579968e00a9b3674/raw/plot2cls.py
%run plot2cls.py

### 実験用のデータ（2次元，2クラス）を生成します．
- Example 1～4 からひとつ選んで実行してください．

In [ ]:
# Example 1: y = or(x1, x2)
X = np.array([[0, 0], [1,0], [0,1], [1,1]])
y = np.array([-1,1,1,1])

X_val = X + np.random.randn(4, 2) * 0.2
y_val = y.copy()

In [ ]:
# Example 2: draw npos and nneg points from the Gaussian distribution for each class
npos = 30
nneg = 30
np.random.seed(321)
X = np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]
# [1,1,...,1,-1,-1,...,-1]
y = np.array([1] * npos + [-1] * nneg)

X_val = X + np.random.randn(X.shape[0], 2) * 1.0
y_val = y.copy()

In [ ]:
# Example 3: create moons using sklearn
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.2, random_state=0)
y[y==0] = -1

X_val, y_val = make_moons(n_samples=100, noise=0.2, random_state=1)
y_val[y_val==0] = -1

In [ ]:
# Example 4: create circles using sklearn
from sklearn.datasets import make_circles
X, y = make_circles(n_samples=150, noise=0.1, random_state=0, factor=0.3)
y[y==0] = -1

X_val, y_val = make_circles(n_samples=150, noise=0.1, random_state=1, factor=0.3)
y_val[y_val==0] = -1

In [ ]:
#@title データを表示します．
plot2cls(X, y, X_val=X_val, y_val=y_val).plot_data()

### ★scikit-learn を用いたロジスティック回帰

[sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)を使用した結果を出力します．

In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()  #(penalty='none')
model.fit(X, y)

# print([w0], [w1, w2])
print("w0 =", model.intercept_, ",  w =", model.coef_.ravel())

print("Accuracy on training data: ", model.score(X, y))
print("Accuracy on validation data: ", model.score(X_val, y_val))

dec = lambda X: model.decision_function(X)
plot2cls(X, y, dec, X_val, y_val).plot_clf()

## 最急降下法によるロジスティック回帰の学習アルゴリズム
参考資料 [■](https://ml-cheatsheet.readthedocs.io/en/latest/logistic_regression.html) [■](https://math.stackexchange.com/questions/78575/derivative-of-sigmoid-function-sigma-x-frac11e-x)

適当な初期値から，学習率 $\alpha>0$ で $w_0$ と $\boldsymbol w$ の修正を次式のように反復します．

$\qquad w_0\gets w_0 - \alpha{\mathit\Delta}w_0$  
$\qquad \boldsymbol w\gets \boldsymbol w - \alpha{\mathit\Delta}\boldsymbol w$

修正量 ${\mathit\Delta}w_0$ と ${\mathit\Delta}\boldsymbol w$ は平均ロジスティック損失 $L(w_0,\boldsymbol w)$ の勾配で，次式のように表せます．

${\mathit\Delta}w_0=\;\;\;\frac{\partial L(w_0,\boldsymbol w)}{\partial w_0}\;\; = \frac{1}{n}\sum_{j=1}^n \varepsilon^{(j)}$  

${\mathit\Delta}\boldsymbol w=\left[\matrix{\frac{\partial L(w_0,\boldsymbol w)}{\partial w_1}\\ \vdots \\\frac{\partial L(w_0,\boldsymbol w)}{\partial w_d}}\right] = \frac{1}{n}\sum_{j=1}^n \varepsilon^{(j)}\boldsymbol x^{(j)}$  

ただし，
$\varepsilon^{(j)}=\left\{\begin{array}{cl}p^{(j)}-1 & (y^{(j)}=+1)\\ p^{(j)} & (y^{(j)}=-1)\end{array}\right.\qquad$  （確率 $p^{(j)}$ を正解 $y^{(j)}$ と比較した誤差）  

$[p^{(1)},\dots,p^{(n)}]=[\mathrm{sigmoid}(g(\boldsymbol x^{(1)})),\dots,\mathrm{sigmoid}(g(\boldsymbol x^{(n)}))]$  

です．つまり，データ $\boldsymbol x^{(j)}$ に対する識別関数の出力を確率 $p^{(j)}$ に換算し，
- 正解が $y^{(j)}=+1$ ならば $p^{(j)}$ が $1$ に近いほどよい（$\varepsilon^{(j)}=p^{(j)}-1$）
- 正解が $y^{(j)}=-1$ ならば $p^{(j)}$ が $0$ に近いほどよい（$\varepsilon^{(j)}=p^{(j)}$）

として誤差 $\varepsilon^{(j)}$ が計算されています．誤差が大きいほど修正量に加味されます．

## ★最急降下法によるロジスティック回帰の実装

LogisticRegressionGDという名のクラスとして実装しましょう．[sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) に似た仕様とします．

訓練データ `X_train` とクラスラベル `y_train` を学習したあと，`X_val` のクラスラベルの予測 `y_pred` を得たり，正答率を表示するには，
```
    myclf = LogisticRegressionGD(lr=1e-2, max_iter=300)
    myclf.fit(X_train, y_train)
    y_pred = myclf.predict(X_val)
    print(myclf.score(X_val, y_val))
```
のように使うことを想定します．ここで，`lr` は学習率（learning rate），`max_iter` は反復回数（maximum number of iterations）です．
訓練データの次元数`d`，個数`n`の場合，`X_train` と `y_train` はそれぞれ shape が `(n,d)` と `(n,)` のNumPy配列です．ただし，`y_train` は2クラスを +1と-1で表すものとします．

In [ ]:
import numpy as np

class LogisticRegressionGD:

    def __init__(self, lr=1., max_iter=1000):
        self.lr = lr
        self.max_iter = max_iter
        self.intercept_ = None                          # w0
        self.coef_ = None                               # [w1,...,wd}
        self.loss_history = []                          # 損失の履歴


    # 識別関数 g(x) = w0 + w1*x1 + ... + wd*xd
    def decision_function(self, X):                     # X(n,d)
        return ''' self.intercept_ と X.dot(self.coef_) を使う '''

    # シグモイド関数 sigmoid(z) = 1/ (1 + exp(-z))
    def _sigmoid(self, z):                              # z(n,)
        return ''' np.exp() を使う '''

    # 平均ロジスティック損失 (1/n)Σ log(1 + exp(-y*g(x)) )
    def ave_loss(self, X, y):                           # X(n,d), y(n,)
        loss = ''' self.decision_function(), np.log(), len(y) を使う '''
        return loss


    # 学習：最急降下法で w0,...,wdを最適化します
    def fit(self, X, y):                                # X(n,d), y(n,)
        n, d = X.shape

        # w0 と [w1,...,wd] の初期化
        if self.intercept_ is None: self.intercept_ = 0
        if self.coef_ is None: self.coef_ = np.zeros(d)

        # 勾配による修正の反復
        for _ in range(self.max_iter):
            p = ''' self.decision_function() と self._sigmoid() を使う．'''
            e = np.where(y > 0, p - 1, p)               # e(n,)

            dw0 = e.sum() / n
            dw = e.dot(X) / n

            self.intercept_ -= ''' self.lr と dw0 を使う '''
            self.coef_ -= ''' self.lr と dw を使う '''

            self.loss_history.append( self.ave_loss(X, y) )


    # 推論
    def predict(self, X):                               # X(n,d)
        y_pred = self.decision_function(X)              # y_pred(n,)
        return np.where(y_pred > 0, 1, -1)              # (n,)

    def score(self, X, y):
        y_pred = self.predict(X)
        return (y == y_pred).sum() / len(y)

### 実装した`LogisticRegressionGD`で識別します．

In [ ]:
model = LogisticRegressionGD()  # lr=100.
model.fit(X, y)

# print([w0], [w1, w2])
print("w0 =", model.intercept_, ",  w =", model.coef_.ravel())

print("Accuracy on training data: ", model.score(X, y))
print("Accuracy on validation data: ", model.score(X_val, y_val))

plt.plot(model.loss_history)

dec = lambda X: model.decision_function(X)
plot2cls(X, y, dec, X_val, y_val).plot_clf()

--------
## 発展課題（任意）

学習率や反復回数を大きくした場合を考えましょう．  
最急降下法によるロジスティック回帰では，同じ問題設定でも scikit-learn の場合と決定境界が異なっていることを確認してください．特に，学習率や反復回数を大きくするほど，求まる重み（`w0` や `[w1,...,wd]`）の値が大きくなります．

    1. 重みの大きさと，平均ロジスティック損失の関係を説明してください．

（ここに回答を書いてください）


ヒント：平均ロジスティック損失 $L(w_0,\boldsymbol w)=\frac{1}{n}\sum_{j=1}^n\log\left(1+e^{-y^{(j)}(w_0+\boldsymbol w^\top\boldsymbol x^{(j)})}\right)$

    2. 「★最急降下法によるロジスティック回帰の実装」を変更してL2正則化を導入し，
       scikit-learn と同じ結果を再現してください．

（コードに変更を施した行の変更前後をここに書いてください）


ヒント：
- L2正則化項を付加した最小化問題： $\mathrm{Minimize}_{(w_0,\boldsymbol w)}\quad L(w_0,\boldsymbol w)+\frac{1}{2}\|\boldsymbol w\|_2^2\qquad$  （ただし，$\|\boldsymbol w\|_2^2=w_1^2+\cdots+w_d^2$）
- L2正則化項 $\frac{1}{2}\|\boldsymbol w\|_2^2$ の勾配は $\boldsymbol w$ なので， $\boldsymbol w$ の修正量 $\mathit\Delta\boldsymbol w$ が $\boldsymbol w$ だけ増える．
- fit を一行変更するだけで済む．$\boldsymbol w$ は `self.coef_` のこと．
- [scikit-learn の最小化問題は `n` で除さない仕様なので](https://scikit-learn.org/stable/modules/linear_model.html#binary-case)，平均ロジスティック損失を採用している場合は `self.coef_ / n` だけ修正量を増やす．


    3. L2正則化を導入したアルゴリズムでは，学習率を大きくすると，重みは大きくならないが，収束しなくなる．  
       このことを確認し，収束しない理由を説明してください．

（ここに回答を書いてください）



お疲れさまでした．